In [1]:
# Load the "autoreload" extension
%load_ext autoreload

# always reload modules marked with "%aimport"
%autoreload 1

import os
import sys

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), os.pardir, 'src', 'data')
sys.path.append(src_dir)

In [2]:
%aimport scrape_buda

In [3]:
from scrape_buda import BudaRating

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
import pymysql as mdb

con = mdb.connect('localhost', 'budauser', 'TrotNixon', 'buda_prod') #host, user, password, 

In [7]:
sqlfile = os.path.join(os.getcwd(), os.pardir, 'data', 'raw', 'buda_prod_backup_2016-08-31.sql')

In [8]:
interim_dir = os.path.join(os.getcwd(), os.pardir, 'data', 'interim')

In [12]:
sqlfile = """SHOW TABLES;"""
buda_tables = pd.read_sql(sqlfile, con)

In [14]:
buda_tables['Tables_in_buda_prod'].values

array(['admin_contact_info', 'application_division_map', 'applications',
       'buda_admin_contact_info', 'buda_admin_positions',
       'buda_admin_positions_map', 'buda_broken_links', 'buda_contacts',
       'buda_devel_documentation', 'buda_groups',
       'buda_hatleague_captain_pwd', 'buda_hatleague_details',
       'buda_hatleague_divisions', 'buda_hatleague_locations',
       'buda_hatleague_player_rankings', 'buda_hatleague_rankings',
       'buda_hatleague_rosters', 'buda_hatleague_rosters_updates',
       'buda_hatleague_schedules', 'buda_hatleague_teams',
       'buda_hs_spirit_scores', 'buda_leagues', 'buda_ll_helpers',
       'buda_ll_schedule', 'buda_ll_volunteers', 'buda_logger',
       'buda_mail_list_requests', 'buda_object_list', 'buda_objects',
       'buda_permissions_map', 'buda_pickup_listings', 'buda_school_teams',
       'buda_sessions', 'buda_team_contact_map', 'buda_team_league_map',
       'buda_teams', 'buda_teams_2003', 'buda_temp_pwd',
       'buda_tourna

In [56]:
sqlfile = """SELECT * FROM joom_jaem_log;"""
buda_teams = pd.read_sql(sqlfile, con)

In [57]:
buda_teams

,id,ext_id,service_id,check_date,check_info
0,1,template-10009,None,2013-11-14 21:46:08,"<div class=\""status-not-uploaded\"">Sorry, no v..."


In [34]:
sqlfile = """SELECT * FROM buda_users_public;"""
buda_users_public = pd.read_sql(sqlfile, con, index_col='user_id')

In [35]:
sqlfile = """SELECT * FROM buda_hatleague_player_rankings;"""
buda_hatleague_player_rankings = pd.read_sql(sqlfile, con)

In [36]:
buda_hatleague_player_rankings = buda_hatleague_player_rankings.rename(columns={'player_id':'user_id'})

In [37]:
buda_hatleague_player_rankings = buda_hatleague_player_rankings.set_index('user_id')

In [38]:
buda_hatleague_player_rankings = buda_hatleague_player_rankings.drop('edited_date', axis=1)

In [39]:
self_ratings = buda_hatleague_player_rankings.join(buda_users_public)

In [40]:
self_ratings[(self_ratings['last_name'] == 'Bussmann') & (self_ratings['first_name'] == 'Shane')]

,league_id,rank_type,status,edited_by,modification_id,rank,experience,other_sport,throwing,break_mark,huck,receiving,height,speed,shirt,recommend_captain,user_name,email,password,first_name,last_name,edited_date,last_login
user_id,,,,,,,,,,,,,,,,,,,,,,,
30295,0,1,NaN,30295,None,49.0,div2,1.0,reliable,sometime,none,both_hands,73.0,fast,large,0.0,sbussmann,sbussmann@gmail.com,6b1888fae695d8e52a01ae5b0fefc2e1:oLVn5pTC3oowU...,Shane,Bussmann,2010-11-18 17:27:34,2016-08-10 19:19:31
30295,30844,1,NaN,3348,None,40.0,None,NaN,None,None,None,None,NaN,None,None,NaN,sbussmann,sbussmann@gmail.com,6b1888fae695d8e52a01ae5b0fefc2e1:oLVn5pTC3oowU...,Shane,Bussmann,2010-11-18 17:27:34,2016-08-10 19:19:31
30295,30919,1,NaN,3424,None,40.0,None,NaN,None,None,None,None,NaN,None,None,NaN,sbussmann,sbussmann@gmail.com,6b1888fae695d8e52a01ae5b0fefc2e1:oLVn5pTC3oowU...,Shane,Bussmann,2010-11-18 17:27:34,2016-08-10 19:19:31
30295,30919,2,NaN,919,None,53.0,hat,1.0,reliable,sometime,short,one_hand,0.0,average,,NaN,sbussmann,sbussmann@gmail.com,6b1888fae695d8e52a01ae5b0fefc2e1:oLVn5pTC3oowU...,Shane,Bussmann,2010-11-18 17:27:34,2016-08-10 19:19:31
30295,30924,1,NaN,10373,None,40.0,None,NaN,None,None,None,None,NaN,None,None,NaN,sbussmann,sbussmann@gmail.com,6b1888fae695d8e52a01ae5b0fefc2e1:oLVn5pTC3oowU...,Shane,Bussmann,2010-11-18 17:27:34,2016-08-10 19:19:31
30295,30924,2,NaN,30295,None,59.0,div2,1.0,reliable,sometime,none,leaping,0.0,fast,,NaN,sbussmann,sbussmann@gmail.com,6b1888fae695d8e52a01ae5b0fefc2e1:oLVn5pTC3oowU...,Shane,Bussmann,2010-11-18 17:27:34,2016-08-10 19:19:31
30295,31034,1,NaN,3424,None,55.0,None,NaN,None,None,None,None,NaN,None,None,NaN,sbussmann,sbussmann@gmail.com,6b1888fae695d8e52a01ae5b0fefc2e1:oLVn5pTC3oowU...,Shane,Bussmann,2010-11-18 17:27:34,2016-08-10 19:19:31
30295,31034,2,NaN,30295,None,66.0,handle_div2,1.0,reliable,sometime,short,one_hand,0.0,fast,,NaN,sbussmann,sbussmann@gmail.com,6b1888fae695d8e52a01ae5b0fefc2e1:oLVn5pTC3oowU...,Shane,Bussmann,2010-11-18 17:27:34,2016-08-10 19:19:31
30295,32741,1,NaN,3348,None,60.0,None,NaN,None,None,None,None,NaN,None,None,NaN,sbussmann,sbussmann@gmail.com,6b1888fae695d8e52a01ae5b0fefc2e1:oLVn5pTC3oowU...,Shane,Bussmann,2010-11-18 17:27:34,2016-08-10 19:19:31


In [41]:
hmm = self_ratings.replace('nan', -1)

In [42]:
len(hmm[hmm['rank'] == -1])

0

In [43]:
sqlfile = """SELECT league_id,league_name,season FROM buda_leagues;"""
buda_leagues = pd.read_sql(sqlfile, con)

In [44]:
self_ratings_league = self_ratings.reset_index().merge(buda_leagues, on='league_id')

In [45]:
len(self_ratings)

47921

In [46]:
len(self_ratings_league)

38296

In [47]:
self_ratings_league = self_ratings_league.drop(['modification_id', 'user_name', 'email', 'password'], axis=1)

In [48]:
self_ratings_league.to_csv(os.path.join(interim_dir, 'self_ratings.csv'))

In [50]:
self_ratings_league[(self_ratings_league['first_name'] == 'Martin')].sort('league_id')

AttributeError: 'DataFrame' object has no attribute 'sort'

In [51]:
ratings.team_players['40328'][2].split(',')[1][1:]

NameError: name 'ratings' is not defined

In [54]:
for league_id in ratings.league_teams:
    if '40328' in ratings.league_teams[league_id]:
        print(league_id)

40258
